In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time, os,requests
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

video_url = 'https://www.instagram.com/emmafituk_/?hl=en'
#create results folder if does not exists
res_folder=os.getcwd()+'/results'
os.makedirs(res_folder, exist_ok=True)
#initiate webdriver
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Optional: run in headless mode
prefs = {"download.default_directory" : res_folder}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument("--headless=new")  # Use the newer headless mode
chrome_options.add_argument("--window-size=1920,1080")  # Set a large enough window size
chrome_options.add_argument("--disable-gpu")  # Sometimes useful in headless
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")


driver = webdriver.Chrome(options=chrome_options)
driver.get('https://fastdl.app/en')
driver.implicitly_wait(10) 

input_element=driver.find_element(By.CSS_SELECTOR,'input')
driver.implicitly_wait(10)
input_element.click()

driver.implicitly_wait(10) 
input_element.send_keys(video_url)
driver.implicitly_wait(10) 
button = driver.find_element(By.CSS_SELECTOR, 'button.search-form__button[type="submit"]')
button.click()
driver.implicitly_wait(10)
time.sleep(5)

body = driver.find_element("tag name", "body")
print(driver.execute_script("return document.body.scrollHeight"))

# Function to scroll down slowly
def smooth_scroll_down(driver, scroll_pause_time=0.2, pixels_per_scroll=100):
  current_scroll = 0  # Keep track of the current scroll position
  while current_scroll < 10000:
    driver.execute_script(f"window.scrollBy(0, {pixels_per_scroll});")
    current_scroll += pixels_per_scroll
    sleep(scroll_pause_time)

# Call the smooth scrolling function
smooth_scroll_down(driver)

print(driver.execute_script("return document.body.scrollHeight"))

time.sleep(5)
buttons = driver.find_elements(By.CSS_SELECTOR, "a.button.button--filled.button__download")
len(buttons)

In [ ]:

driver.quit()


In [ ]:
buttons[0].get_attribute('href')

In [ ]:
#import img in base 64
import base64
image_path = 'results/frames_DC7PCgOoF2t_2/DC7PCgOoF2t_0.jpg'
with open(image_path, "rb") as image_file:
    base64_string = base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
base64_string

In [ ]:

# Get list of all .jpg files in the directory
import os
directory = 'results/frames_DC7PCgOoF2t_2'
jpg_files = [file for file in os.listdir(directory) if file.endswith(".jpg")]

print("JPG files:", jpg_files)

In [ ]:
from utils.download_content import download_content
from utils.coach_ig_info import coach_ig_info
from utils.mp4_to_jpg import mp4_to_jpg
from utils.img_interpreter import img_interpreter
import os, shutil,base64


#specify the video url
coach_url = 'https://www.instagram.com/emmafituk_/?hl=en'
posts_info = coach_ig_info(coach_url.split('/')[-2])

In [ ]:
posts_info._date

In [ ]:
import subprocess, os, shutil
from moviepy.editor import VideoFileClip
#compute video duration
video_name='DC4_BEyRwbA_0.mp4'
video_filename = 'results/'+video_name
video_filename

In [ ]:
def mp4_to_jpg(video_name = 'DDDEwC4pVWm_0.mp4'):
  from moviepy.editor import VideoFileClip
  import subprocess, os, shutil

  #clean old results
  directory="results/frames_"+video_name[:-6]
  if os.path.exists(directory): shutil.rmtree(directory)
  os.mkdir(directory)

  video_filename = 'results/'+video_name

  #compute video duration
  video = VideoFileClip(video_filename)
  duration = video.duration
  #compute the frames seconds based on the video length
  seconds = [f"00:00:{x:02}" for x in [round(1 + i * (duration - 1) / 5) for i in range(6)]]

  for x,second in enumerate(seconds):
      command = 'ffmpeg -y -ss '+str(second)+' -i  '+video_filename +' -frames:v 1 '+directory+'/frame_'+str(x)+'.jpg'
      subprocess.run(command, shell=True, executable="/bin/bash",stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
      
  return directory

In [ ]:
mp4_to_jpg(video_name='DC4_BEyRwbA_0.mp4')

In [ ]:
os.remove('results/DC4_BEyRwbA_0.mp4')

In [ ]:
directory = 'results/frames_DC4_BEyRwbA'
images=[file for file in os.listdir(directory) if file.endswith(".jpg")]
images

In [ ]:
images_description = 'Summaries of the descriptions of the images in DC4_BEyRwbA content:\n\n'
for i,ip in enumerate(images):
    image_path=directory+'/'+ip
    with open(image_path, "rb") as image_file: base64_string = base64.b64encode(image_file.read()).decode('utf-8')
    img_description = img_interpreter(base64_string)
    images_description = images_description + 'Image_'+str(i)+ ':' +img_description+'\n\n'
    #delete the image
    os.remove(image_path)

In [ ]:
def download_content(video_url,coach_handel):
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    import time, os,requests
    from bs4 import BeautifulSoup


    #create results folder if does not exists
    content_name = video_url.split('/')[-2]
    res_folder=os.getcwd()+'/results/'+str(coach_handel)+'/'+str(content_name)
    if os.path.exists(res_folder) and os.path.isdir(res_folder): shutil.rmtree(res_folder)
    os.makedirs(res_folder, exist_ok=True)
    #initiate webdriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Optional: run in headless mode
    prefs = {"download.default_directory" : res_folder}
    chrome_options.add_experimental_option("prefs",prefs)

    driver = webdriver.Chrome(options=chrome_options)
    driver.get('https://fastdl.app/en')
    driver.implicitly_wait(2) 
    
    while True:
        try:
            input_element=driver.find_element(By.CSS_SELECTOR,'input')
            driver.implicitly_wait(3)
            input_element.click()

            driver.implicitly_wait(3) 
            input_element.send_keys(video_url)
            driver.implicitly_wait(3) 
            button = driver.find_element(By.CSS_SELECTOR, 'button.search-form__button[type="submit"]')
            button.click()
            driver.implicitly_wait(3)
            time.sleep(2)
            break
        except:
            time.sleep(0.5)
            pass

    while True:
        try:
            buttons = driver.find_elements(By.CSS_SELECTOR, "a.button.button--filled.button__download")
            button_list=[]
            for button in buttons:
                driver.implicitly_wait(5) 
                button_list.append(button.get_attribute('href'))
            for i,url in enumerate(button_list):
                if '.mp4' in url: extension='.mp4'
                else: extension='.jpg'
                r = requests.get(url)
                results_directory=res_folder+'/'+str(content_name)+'_'+str(i)+extension
                with open(results_directory, 'wb') as f:
                    f.write(r.content)
            Last_created_file=str(content_name)+'_'+str(i)+extension
            if Last_created_file in [f for f in os.listdir(res_folder) if os.path.isfile(os.path.join(res_folder, f))]:
                break
        except:
            time.sleep(0.5)
            pass


    driver.quit()
    files = [res_folder+'/'+f for f in os.listdir(res_folder) if os.path.isfile(os.path.join(res_folder, f))]
    return files

file_names=download_content(video_url='https://www.instagram.com/p/DDAVJ0DoyVc/?hl=en',coach_handel='emmafituk_')

In [ ]:
if '.mp4' in file_names[0]:
    print('ok!')

In [ ]:
file_names[0]


In [ ]:
directory=''
for i,f in enumerate(file_names[0].split('/')):
    if i != len(file_names[0].split('/'))-1:
        directory=directory+f+'/'
directory[:-1]

In [ ]:
from utils.coach_ig_info import coach_handles_all

coach_handles_all_l=coach_handles_all()
coach_urls = ['https://www.instagram.com/'+str(h)+'/?hl=en' for h in coach_handles_all_l.handle]

In [ ]:
coach_urls